In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,8868
2,Huelva,Confirmados PDIA 14 días,2413
3,Huelva,Tasa PDIA 14 días,"472,4489616108297"
4,Huelva,Confirmados PDIA 7 días,1049
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,7
629,Municipio de Huelva sin especificar,Total Confirmados,110
630,Municipio de Huelva sin especificar,Curados,52


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  8868.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2487.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 428 personas en los últimos 7 días 

Un positivo PCR cada 203 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,8868.0,1049.0,2413.0,510743.0,205.387054,472.448962,84.0
Huelva-Costa,4804.0,666.0,1441.0,288115.0,231.157697,500.147511,56.0
Huelva (capital),2487.0,336.0,706.0,143663.0,233.880679,491.427855,31.0
Condado-Campiña,2762.0,316.0,730.0,155057.0,203.796023,470.794611,18.0
Lepe,762.0,120.0,245.0,27431.0,437.461266,893.150086,9.0
Sierra de Huelva-Andévalo Central,1193.0,60.0,233.0,67571.0,88.795489,344.822483,8.0
Niebla,94.0,14.0,26.0,4117.0,340.053437,631.527812,6.0
Aljaraque,325.0,35.0,81.0,21260.0,164.628410,380.997178,6.0
Isla Cristina,252.0,33.0,81.0,21264.0,155.191874,380.925508,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,17.0,6.0,14.0,548.0,1094.890511,2554.744526,0.0
Palma del Condado (La),399.0,44.0,102.0,10761.0,408.883933,947.867299,2.0
Lepe,762.0,120.0,245.0,27431.0,437.461266,893.150086,9.0
Villarrasa,88.0,2.0,19.0,2176.0,91.911765,873.161765,0.0
Lucena del Puerto,55.0,15.0,29.0,3371.0,444.971818,860.278849,1.0
Minas de Riotinto,60.0,10.0,29.0,3848.0,259.875260,753.638254,2.0
Villablanca,54.0,5.0,20.0,2848.0,175.561798,702.247191,0.0
Palos de la Frontera,166.0,23.0,77.0,11289.0,203.738152,682.079901,1.0
Campillo (El),52.0,2.0,13.0,2023.0,98.863075,642.609985,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,257.0,6.0,58.0,12820.0,46.801872,452.418097,0.0,0.103448
Villarrasa,88.0,2.0,19.0,2176.0,91.911765,873.161765,0.0,0.105263
San Juan del Puerto,131.0,3.0,25.0,9300.0,32.258065,268.817204,0.0,0.120000
Aracena,278.0,5.0,35.0,8107.0,61.675096,431.725669,0.0,0.142857
Paterna del Campo,50.0,1.0,7.0,3509.0,28.498148,199.487033,0.0,0.142857
Campillo (El),52.0,2.0,13.0,2023.0,98.863075,642.609985,1.0,0.153846
Jabugo,34.0,2.0,12.0,2250.0,88.888889,533.333333,0.0,0.166667
Villablanca,54.0,5.0,20.0,2848.0,175.561798,702.247191,0.0,0.250000
Nerva,74.0,4.0,16.0,5235.0,76.408787,305.635148,0.0,0.250000
